# Goal: showcase results of the V1 experiments with gradual causal ABA

The v1 experiment uses original causal aba framework but with additional rules.

In [ ]:
import pandas as pd
other_methods_data = pd.read_csv('../../results/extension_based_semantics/stored_results_bnlearn_50rep_cpdag.csv')
v1_data = pd.read_csv('../../results/gradual/experiment_v1/cpdag_metrics.csv')
only_gradual_v1_data = pd.read_csv('../../results/gradual/stand_alone_gradual_v1/cpdag_metrics.csv')

In [2]:
v1_data.head()

,nnz,fdr,tpr,fpr,precision,recall,F1,shd,sid_low,sid_high,...,seed,n_nodes,elapsed_bsaf_creation,elapsed_model_solution,is_converged,fact_ranking_method,model_ranking_method,aba_elapsed,ranking_elapsed,best_I
0,4,0.2500,0.75,0.1667,0.7500,0.75,0.7500,1.0,5.0,10.0,...,7816,5,12.234522,0.907921,True,original,original,0.677906,0.054963,21.402040
1,4,0.2500,0.75,0.1667,0.7500,0.75,0.7500,1.0,5.0,10.0,...,7816,5,12.234522,0.907921,True,original,refined_indep_facts,0.677906,0.037848,2.212397
2,7,0.5714,0.75,0.6667,0.2857,0.50,0.3636,5.0,8.0,14.0,...,7816,5,12.234522,0.907921,True,original,arrows_sum,0.677906,0.022329,3.514415
3,4,0.2500,0.75,0.1667,0.7500,0.75,0.7500,1.0,5.0,10.0,...,7816,5,12.234522,0.907921,True,original,arrows_mean,0.677906,0.022565,0.508232
4,2,0.5000,0.25,0.1667,0.5000,0.25,0.3333,3.0,10.0,10.0,...,7816,5,12.234522,0.907921,True,refined_indep_facts,original,0.022879,0.025342,7.341826


In [3]:
only_gradual_v1_data

,nnz,fdr,tpr,fpr,precision,recall,F1,shd,sid_low,sid_high,...,seed,n_nodes,elapsed_bsaf_creation,elapsed_model_solution,is_converged,fact_ranking_method,model_ranking_method,aba_elapsed,ranking_elapsed,best_I
0,4,0.5000,0.5000,0.3333,0.5000,0.5000,0.5000,2.0,10.0,10.0,...,7816,5,12.224487,0.915898,True,stand_alone_gradual,original_ranking,0,1.346822,21.245104
1,0,0.0000,0.0000,0.0000,NaN,0.0000,NaN,4.0,10.0,10.0,...,7816,5,12.224487,0.915898,True,stand_alone_gradual,refined_ranking,0,0.531603,3.967036
2,4,0.5000,0.5000,0.3333,0.5000,0.5000,0.5000,2.0,10.0,10.0,...,3578,5,12.224487,0.918503,True,stand_alone_gradual,original_ranking,0,1.330360,20.714498
3,0,0.0000,0.0000,0.0000,NaN,0.0000,NaN,4.0,10.0,10.0,...,3578,5,12.224487,0.918503,True,stand_alone_gradual,refined_ranking,0,0.550066,4.363300
4,3,0.6667,0.2500,0.3333,0.3333,0.2500,0.2857,3.0,11.0,11.0,...,2656,5,12.224487,0.918005,True,stand_alone_gradual,original_ranking,0,0.616465,12.970899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0,0.0000,0.0000,0.0000,NaN,0.0000,NaN,6.0,17.0,17.0,...,4125,6,1991.199507,18.155988,True,stand_alone_gradual,refined_ranking,0,0.859803,9.616728
296,4,0.5000,0.3333,0.2222,0.5000,0.3333,0.4000,6.0,16.0,23.0,...,3094,6,1991.199507,18.128719,True,stand_alone_gradual,original_ranking,0,1.374774,12.252958
297,0,0.0000,0.0000,0.0000,NaN,0.0000,NaN,6.0,17.0,17.0,...,3094,6,1991.199507,18.128719,True,stand_alone_gradual,refined_ranking,0,0.901448,9.286791
298,2,0.0000,0.3333,0.0000,1.0000,0.3333,0.5000,5.0,11.0,19.0,...,6398,6,1991.199507,18.034173,True,stand_alone_gradual,original_ranking,0,0.869354,13.406162


In [4]:
DAG_ARCS_MAP = {'asia':8, 'cancer':4, 'earthquake':4, 'sachs':17, 'survey':6, 'alarm':46, 'child':25, 'insurance':52, 'hailfinder':66, 'hepar2':123}
DAG_NODES_MAP = {'asia':8, 'cancer':5, 'earthquake':5, 'sachs':11, 'survey':6, 'alarm':37, 'child':20, 'insurance':27, 'hailfinder':56, 'hepar2':70}

v1_data['model'] = 'f__' + v1_data['fact_ranking_method'] + '__r__' + v1_data['model_ranking_method']
only_gradual_v1_data['model'] = 'only_gradual__' + only_gradual_v1_data['model_ranking_method']

def process_v1_data(df):
    groupby_cols = ['dataset', 'n_nodes', 'model']
    df_grouped = df.groupby(groupby_cols, as_index=False).aggregate(
        sid_low_mean=('sid_low', 'mean'),
        sid_high_mean=('sid_high', 'mean'),
        sid_low_std=('sid_low', 'std'),
        sid_high_std=('sid_high', 'std'),
    )
    df_grouped['n_edges'] = df_grouped['dataset'].map(DAG_ARCS_MAP)
    df_grouped['p_SID_low_mean'] = df_grouped['sid_low_mean'] / df_grouped['n_edges']
    df_grouped['p_SID_high_mean'] = df_grouped['sid_high_mean'] / df_grouped['n_edges']
    df_grouped['p_SID_low_std'] = df_grouped['sid_low_std'] / df_grouped['n_edges']
    df_grouped['p_SID_high_std'] = df_grouped['sid_high_std'] / df_grouped['n_edges']
    return df_grouped

v1_grouped = process_v1_data(v1_data)
only_gradual_v1_grouped = process_v1_data(only_gradual_v1_data)

only_gradual_v1_grouped.head()

,dataset,n_nodes,model,sid_low_mean,sid_high_mean,sid_low_std,sid_high_std,n_edges,p_SID_low_mean,p_SID_high_mean,p_SID_low_std,p_SID_high_std
0,cancer,5,only_gradual__original_ranking,10.06,10.66,0.651842,1.422989,4,2.515000,2.665,0.162960,0.355747
1,cancer,5,only_gradual__refined_ranking,10.00,10.00,0.000000,0.000000,4,2.500000,2.500,0.000000,0.000000
2,earthquake,5,only_gradual__original_ranking,3.30,13.14,3.587108,4.899021,4,0.825000,3.285,0.896777,1.224755
3,earthquake,5,only_gradual__refined_ranking,10.00,10.00,0.000000,0.000000,4,2.500000,2.500,0.000000,0.000000
4,survey,6,only_gradual__original_ranking,13.52,18.54,2.643745,3.144220,6,2.253333,3.090,0.440624,0.524037


In [5]:
other_methods_data.head()

,dataset,model,elapsed_mean,elapsed_std,nnz_mean,nnz_std,fdr_mean,fdr_std,tpr_mean,tpr_std,...,recall_mean,recall_std,F1_mean,F1_std,shd_mean,shd_std,sid_low_mean,sid_low_std,sid_high_mean,sid_high_std
0,cancer,ABAPC (ASPforABA),0.47,0.24,3.86,0.64,0.53,0.11,0.46,0.13,...,0.44,0.15,0.47,0.11,2.34,0.69,9.74,1.48,10.64,1.54
1,cancer,Random,0.00,0.00,4.00,0.00,0.70,0.24,0.30,0.24,...,0.20,0.19,0.32,0.13,5.72,1.55,7.94,3.63,14.04,3.03
2,cancer,MPC,0.00,0.00,3.76,0.52,0.01,0.04,0.93,0.11,...,0.50,0.00,0.52,0.04,2.04,0.20,3.88,1.81,13.88,1.81
3,cancer,ABAPC (Ours),0.96,1.00,3.86,0.64,0.53,0.11,0.46,0.13,...,0.44,0.15,0.47,0.11,2.34,0.69,9.74,1.48,10.64,1.54
4,cancer,NOTEARS-MLP,5.69,3.72,1.92,0.49,0.43,0.20,0.25,0.00,...,0.25,0.00,0.34,0.03,3.00,0.00,9.44,1.54,10.24,0.43


In [6]:
other_methods_data['n_edges'] = other_methods_data['dataset'].map(DAG_ARCS_MAP)
other_methods_data['n_nodes'] = other_methods_data['dataset'].map(DAG_NODES_MAP)
other_methods_data['p_SID_low_mean'] = other_methods_data['sid_low_mean'] / other_methods_data['n_edges']
other_methods_data['p_SID_high_mean'] = other_methods_data['sid_high_mean'] / other_methods_data['n_edges']
other_methods_data['p_SID_low_std'] = other_methods_data['sid_low_std'] / other_methods_data['n_edges']
other_methods_data['p_SID_high_std'] = other_methods_data['sid_high_std'] / other_methods_data['n_edges']
other_methods_data.head()

,dataset,model,elapsed_mean,elapsed_std,nnz_mean,nnz_std,fdr_mean,fdr_std,tpr_mean,tpr_std,...,sid_low_mean,sid_low_std,sid_high_mean,sid_high_std,n_edges,n_nodes,p_SID_low_mean,p_SID_high_mean,p_SID_low_std,p_SID_high_std
0,cancer,ABAPC (ASPforABA),0.47,0.24,3.86,0.64,0.53,0.11,0.46,0.13,...,9.74,1.48,10.64,1.54,4,5,2.435,2.66,0.3700,0.3850
1,cancer,Random,0.00,0.00,4.00,0.00,0.70,0.24,0.30,0.24,...,7.94,3.63,14.04,3.03,4,5,1.985,3.51,0.9075,0.7575
2,cancer,MPC,0.00,0.00,3.76,0.52,0.01,0.04,0.93,0.11,...,3.88,1.81,13.88,1.81,4,5,0.970,3.47,0.4525,0.4525
3,cancer,ABAPC (Ours),0.96,1.00,3.86,0.64,0.53,0.11,0.46,0.13,...,9.74,1.48,10.64,1.54,4,5,2.435,2.66,0.3700,0.3850
4,cancer,NOTEARS-MLP,5.69,3.72,1.92,0.49,0.43,0.20,0.25,0.00,...,9.44,1.54,10.24,0.43,4,5,2.360,2.56,0.3850,0.1075


In [7]:
data_to_plot_columns = ['dataset', 'n_nodes', 'n_edges', 'model', 'p_SID_low_mean', 'p_SID_high_mean', 'p_SID_low_std', 'p_SID_high_std']

data_to_plot = pd.concat([other_methods_data[data_to_plot_columns], 
                         v1_grouped[data_to_plot_columns], 
                         only_gradual_v1_grouped[data_to_plot_columns]], ignore_index=True)

data_to_plot.head()

,dataset,n_nodes,n_edges,model,p_SID_low_mean,p_SID_high_mean,p_SID_low_std,p_SID_high_std
0,cancer,5,4,ABAPC (ASPforABA),2.435,2.66,0.3700,0.3850
1,cancer,5,4,Random,1.985,3.51,0.9075,0.7575
2,cancer,5,4,MPC,0.970,3.47,0.4525,0.4525
3,cancer,5,4,ABAPC (Ours),2.435,2.66,0.3700,0.3850
4,cancer,5,4,NOTEARS-MLP,2.360,2.56,0.3850,0.1075


In [8]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

def plot_sid_grouped_by_type(df: pd.DataFrame, model_order: list[str]):
    """
    Plot grouped bar chart showing Parent SID (Low/High) for each model, with spacing between groups.

    Args:
        df (pd.DataFrame): DataFrame for one dataset with columns:
            - dataset, n_nodes, n_edges, model
            - p_SID_low_mean, p_SID_high_mean
            - p_SID_low_std, p_SID_high_std
        model_order (list[str]): List of models in desired order
    """
    # Ensure only one dataset
    dataset_labels = df.apply(lambda row: f"{row['dataset']} |V|={row['n_nodes']}, |E|={row['n_edges']}", axis=1).unique()
    if len(dataset_labels) != 1:
        raise ValueError("Function only supports a single dataset.")
    dataset_label = dataset_labels[0]

    # Melt into long format
    df_low = df[["model", "p_SID_low_mean", "p_SID_low_std"]].copy()
    df_low["SID_type"] = "Low"
    df_low.rename(columns={"p_SID_low_mean": "SID_mean", "p_SID_low_std": "SID_std"}, inplace=True)

    df_high = df[["model", "p_SID_high_mean", "p_SID_high_std"]].copy()
    df_high["SID_type"] = "High"
    df_high.rename(columns={"p_SID_high_mean": "SID_mean", "p_SID_high_std": "SID_std"}, inplace=True)

    df_long = pd.concat([df_low, df_high], ignore_index=True)
    df_long = df_long[df_long["model"].isin(model_order)].copy()
    df_long["model"] = pd.Categorical(df_long["model"], categories=model_order, ordered=True)

    # Colors
    palette = px.colors.qualitative.Set2 + px.colors.qualitative.Plotly
    model_colors = {model: palette[i % len(palette)] for i, model in enumerate(model_order)}

    # Plotting positions
    n_models = len(model_order)
    group_spacing = n_models + 1  # space between Low and High
    x = []
    y = []
    errors = []
    colors = []
    model_names = []

    for sid_group_idx, sid_type in enumerate(["Low", "High"]):
        base = sid_group_idx * group_spacing
        for model_idx, model in enumerate(model_order):
            xpos = base + model_idx
            row = df_long[(df_long["model"] == model) & (df_long["SID_type"] == sid_type)]
            if not row.empty:
                x.append(xpos)
                y.append(row["SID_mean"].values[0])
                errors.append(row["SID_std"].values[0])
                colors.append(model_colors[model])
                model_names.append(model)

    # Main bars (no text labels)
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=x,
        y=y,
        error_y=dict(type='data', array=errors, visible=True),
        marker_color=colors,
        hovertemplate="Model: %{customdata}<br>SID: %{y:.2f}<extra></extra>",
        customdata=model_names,
        showlegend=False
    ))

    # Manual legend
    for model in model_order:
        fig.add_trace(go.Bar(
            x=[None], y=[None],
            marker_color=model_colors[model],
            name=model,
            showlegend=True
        ))

    # X-axis tick labels centered
    tick_positions = [
        (0 + (n_models - 1) / 2),
        (group_spacing + (n_models - 1) / 2)
    ]
    tick_labels = ["Low", "High"]

    fig.update_layout(
        title=f"Parent SID (Low vs High) — {dataset_label}",
        xaxis=dict(
            tickmode='array',
            tickvals=tick_positions,
            ticktext=tick_labels,
            title="SID Type"
        ),
        yaxis_title="Parent SID",
        bargap=0,
        showlegend=True,
        legend_title="Model",
        height=800  # ← Increase this value to make the figure taller
    )

    fig.show()


In [9]:
data_to_plot.model.unique()

array(['ABAPC (ASPforABA)', 'Random', 'MPC', 'ABAPC (Ours)',
       'NOTEARS-MLP', 'f__original__r__arrows_mean',
       'f__original__r__arrows_sum', 'f__original__r__original',
       'f__original__r__refined_indep_facts',
       'f__refined_indep_facts__r__arrows_mean',
       'f__refined_indep_facts__r__arrows_sum',
       'f__refined_indep_facts__r__original',
       'f__refined_indep_facts__r__refined_indep_facts',
       'only_gradual__original_ranking', 'only_gradual__refined_ranking'],
      dtype=object)

In [10]:
model_order=['Random', 'NOTEARS-MLP', 'MPC',
            'f__original__r__original',
            'only_gradual__original_ranking',
            'f__original__r__refined_indep_facts',
            'f__original__r__arrows_sum', 
            'f__original__r__arrows_mean',
            'f__refined_indep_facts__r__original',
            'f__refined_indep_facts__r__refined_indep_facts',
            'f__refined_indep_facts__r__arrows_sum',
            'f__refined_indep_facts__r__arrows_mean']

In [11]:
plot_sid_grouped_by_type(data_to_plot[data_to_plot['dataset']=='cancer'],
                    model_order=model_order)

In [12]:
plot_sid_grouped_by_type(data_to_plot[data_to_plot['dataset']=='earthquake'],
                    model_order=model_order)

In [13]:
plot_sid_grouped_by_type(data_to_plot[data_to_plot['dataset']=='survey'],
                    model_order=model_order)

# Conclusion

No improvement observed when using refined independence facts. Likely because we missed out on dependence information.
However arrows mean and arrows sum ranking provides good results. This means that the arrow strengths do contain meaningful info.
This experiment is stepping stone for out standalone gradual-aba experiment.